 # Min-Max Ant System per Maximum Flow Problem

 Questo notebook implementa l'algoritmo **Min-Max Ant System (MMAS)** per risolvere il problema del **Maximum Flow** su grafi diretti.

## Caratteristiche principali del MMAS:
 ✅ Solo la **miglior formica** può depositare feromone

 ✅ Limiti **τ_min** e **τ_max** per evitare stagnazione  

 ✅ Inizializzazione con **τ_max** per favorire esplorazione
 
 ✅ Evaporazione controllata per convergenza stabile

## 1. Import e Configurazione Parametri dell'Algoritmo MMAS

In [ ]:
import networkx as nx
import matplotlib
import numpy as np
import statistics
import random


import matplotlib.pyplot as plt

alpha = 1.0  # controlla l'importanza del pheromone
beta = 2.0  # controlla l'importanza di eta nel nostro caso la capacità
rho = 0.1  # tasso di evaporazione del feromone (più basso per MMAS)
Q = 100.0  # costante per il calcolo del deposito di feromone

# Parametri specifici Min-Max Ant System
tau_max = 10.0  # valore massimo di feromone
tau_min = 0.01  # valore minimo di feromone

print("🎯 Parametri MMAS configurati:")
print(f"   • α (importanza feromone): {alpha}")
print(f"   • β (importanza euristica): {beta}")
print(f"   • ρ (evaporazione): {rho}")
print(f"   • τ_min: {tau_min}")
print(f"   • τ_max: {tau_max}")
print(f"   • Q (deposito): {Q}")

## 2. Funzioni di Utilità per il Grafo

In [ ]:
# === Calcolo capacità uscenti ===
def somma_capacita_uscenti(G, nodo):
    return sum(G[u][v]["capacity"] for u, v in G.out_edges(nodo))
    """
     In un contesto MMAS, conoscere la capacità totale in uscita da un nodo
     (es. la sorgente) è utile per stimare quanto flusso può essere inviato 
     attraverso la rete a partire da quel nodo. Serve come limite superiore locale.
     """

# === Calcolo capacità entranti ===
def somma_capacita_entranti(G, nodo):
    return sum(G[u][v]["capacity"] for u, v in G.in_edges(nodo))
    """
    Analogamente, la capacità totale entrante in un nodo (es. il pozzo)
    indica quanto flusso può essere ricevuto complessivamente da quel nodo.
    In MMAS, questo valore aiuta a valutare la qualità o la validità delle soluzioni candidate.
    """


# === Calcolo flusso massimo teorico ===
def calcola_flusso_massimo(G, sorgente, pozzo):
    sorgente_sum = somma_capacita_uscenti(G, sorgente)
    pozzo_sum = somma_capacita_entranti(G, pozzo)
    return min(sorgente_sum, pozzo_sum)
    """
    In MMAS, è fondamentale avere una stima del flusso massimo teorico
    per guidare l’evaporazione e il rafforzamento dei feromoni.
    Il valore minimo tra la capacità uscente dalla sorgente e quella entrante nel pozzo
    rappresenta un limite superiore teorico del flusso massimo ottenibile nella rete.
    """

## 3. Inizializzazione del Grafo dalla Rete

In [ ]:
# === Lettura file e costruzione grafo ===
def inizializzazione_grafo():
    G = nx.DiGraph()
    
    ###INSERIRE L'ISTANZA DA ESEGUIRE#### 
    #esempio file_path = r"\progetto\istanze\network_160.txt"
    
    import os
    nome_file = os.path.basename(file_path)
    nome_rete = nome_file.replace('.txt', '')  # Rimuovi estensione

    with open(file_path, "r") as file:
        nodi = int(file.readline())
        archi = int(file.readline())
        sorgente = int(file.readline())
        pozzo = int(file.readline())

        print(f"Numero Nodi: {nodi}, Numero Archi: {archi}")
        print(f"Sorgente: {sorgente}, Pozzo: {pozzo}")

        for _ in range(archi):
            u, v, peso = file.readline().split()
            # MMAS: Inizializza tutti gli archi con tau_max per favorire l'esplorazione
            G.add_edge(
                int(u), int(v), capacity=float(peso), pheromone=tau_max, eta=float(peso)
            )

    return G, sorgente, pozzo, nome_rete

## 4. Regola di Transizione Probabilistica

In [ ]:
def proportional_transition_rule(G, u, alpha=1.0, beta=2.0):
    vettore_prob = []

    vicini = list(G.successors(u))
    if not vicini:
        return []  # nessun vicino, nessuna transizione possibile

    somma = 0.0
    # Calcola il denominatore della formula
    for v in vicini:
        tau = G[u][v]["pheromone"]
        eta = G[u][v]["eta"]
        somma += (tau**alpha) * (eta**beta)

    # Calcola le probabilità individuali
    for v in vicini:
        tau = G[u][v]["pheromone"]
        eta = G[u][v]["eta"]
        prob = (tau**alpha) * (eta**beta) / somma
        nodo = {"nodo_uscente": v, "probabilità": prob}
        vettore_prob.append(nodo)

    return vettore_prob

## 5. Funzione di Visualizzazione 

In [ ]:
def disegna_grafico_flusso_multi_run(tutti_flussi_iterazione, flusso_max_teorico, statistiche, nome_rete):
    """
    Disegna un grafico ottimizzato per la leggibilità con curve ben separate
    e visualizzazione chiara del numero di iterazioni per ogni run.
    """
    # Figura più grande per migliore leggibilità
    plt.figure(figsize=(16, 10))
    plt.style.use("seaborn-v0_8-whitegrid")

    # Usiamo colori differenti per distinguere le runs 
    colori = [
        "#1f77b4",  # blu
        "#ff7f0e",  # arancione
        "#2ca02c",  # verde
        "#d62728",  # rosso
        "#9467bd",  # viola
        "#8c564b",  # marrone
        "#e377c2",  # rosa
        "#7f7f7f",  # grigio
        "#bcbd22",  # oliva
        "#17becf",  # ciano
    ]

    iterazioni_teorico = []
    for flussi in tutti_flussi_iterazione:
        try:
            idx = next(
                i for i, f in enumerate(flussi) if abs(f - flusso_max_teorico) < 1e-6
            )
        except StopIteration:
            idx = None
        iterazioni_teorico.append(idx)

    # Ricerca della run migliore con la convergenza più veloce ed il minor numero di iterazioni
    run_migliore = None
    iter_migliore = float("inf")
    for idx, iter_raggiunta in enumerate(iterazioni_teorico):
        if iter_raggiunta is not None and iter_raggiunta < iter_migliore:
            iter_migliore = iter_raggiunta
            run_migliore = idx

    # Trova la lunghezza massima per scala x appropriata
    max_iterazioni = max(len(flussi) for flussi in tutti_flussi_iterazione)
    
    # Calcola le statistiche delle iterazioni qui per averle disponibili
    iterazioni_valide = [it for it in iterazioni_teorico if it is not None]
    if iterazioni_valide:
        min_iter = min(iterazioni_valide)
        max_iter = max(iterazioni_valide)
        avg_iter = sum(iterazioni_valide) / len(iterazioni_valide)
    else:
        min_iter = max_iter = avg_iter = 0
    
    # Plotta ogni run con stile distinto
    for i, flussi in enumerate(tutti_flussi_iterazione):
        iterazioni = list(range(len(flussi)))
        is_best = i == run_migliore
        
        # Stile per evidenziare il run migliore
        lw = 3.5 if is_best else 2.0
        col = "#000000" if is_best else colori[i % len(colori)]
        alpha = 1.0 if is_best else 0.8
        zorder = 10 if is_best else 5
        
        # Etichetta per la legenda con numero di iterazioni ben visibile
        iter_raggiunta = iterazioni_teorico[i]
        if iter_raggiunta is not None:
            label_iter = f"{iter_raggiunta}"
            if is_best:
                label = f"* Run {i+1} -> {label_iter} iter (BEST)"
            else:
                label = f"Run {i+1} -> {label_iter} iter (OK)" 
        else:
            label = f"Run {i+1} -> NC (Non Convergente)"

        # Plot con marcatori per migliore visibilità
        plt.plot(iterazioni, flussi, 
                color=col, 
                linewidth=lw, 
                alpha=alpha,
                label=label,
                zorder=zorder,
                marker='o' if is_best else None,
                markersize=4 if is_best else 0,
                markevery=max(1, len(iterazioni)//15) if is_best else 0)
        
        # Aggiungi annotazione per il punto di convergenza
        if iter_raggiunta is not None and iter_raggiunta < len(flussi):
            plt.annotate(f'{iter_raggiunta}', 
                        xy=(iter_raggiunta, flussi[iter_raggiunta]),
                        xytext=(10, 10), 
                        textcoords='offset points',
                        bbox=dict(boxstyle='round,pad=0.3', 
                                 facecolor='yellow' if is_best else 'white', 
                                 alpha=0.8,
                                 edgecolor=col),
                        fontsize=10,
                        fontweight='bold' if is_best else 'normal',
                        ha='left')

    # Linea flusso teorico più prominente
    plt.axhline(
        y=flusso_max_teorico,
        color="red",
        linestyle="--",
        linewidth=3,
        alpha=0.9,
        label=f"Flusso Teorico: {flusso_max_teorico:.0f}",
        zorder=15
    )

    # Migliora scala e assi
    plt.xlabel("Iterazione", fontsize=16, fontweight="bold")
    plt.ylabel("Flusso", fontsize=16, fontweight="bold")
    plt.title(f"Min-Max Ant System - Convergenza Multi-Run ({nome_rete})\n(Numero iterazioni mostrato per ogni run)", 
              fontsize=18, fontweight="bold", pad=25)

    # Scala Y ottimizzata per vedere meglio la convergenza
    min_flusso = min(min(flussi) for flussi in tutti_flussi_iterazione)
    plt.ylim(min_flusso * 0.95, flusso_max_teorico * 1.02)
    
    # Scala X con margine
    plt.xlim(-max_iterazioni * 0.02, max_iterazioni * 1.05)

    # Griglia più leggera per non disturbare
    plt.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)

    # Legenda migliorata con due colonne
    plt.legend(fontsize=12, 
              loc="center right", 
              bbox_to_anchor=(1.3, 0.5),
              ncol=1,
              frameon=True,
              fancybox=True,
              shadow=True,
              title="Run e Iterazioni",
              title_fontsize=14)

    # Box con statistiche posizionato in alto a destra
    stats_text = f"""Iterazioni:
Min: {min_iter:.0f} iter
Max: {max_iter:.0f} iter  
Avg: {avg_iter:.0f} iter
Convergenti: {len(iterazioni_valide)}/10

Statistiche MMAS:
Flusso Best: {statistiche['best']:.0f}
Flusso Mean: {statistiche['mean']:.1f}
Flusso Std: {statistiche['std_dev']:.1f}"""

    plt.text(0.98, 0.02, stats_text,
            transform=plt.gca().transAxes,
            verticalalignment='bottom',
            horizontalalignment='right',
            bbox=dict(boxstyle="round,pad=0.6", 
                     facecolor="lightcyan", 
                     alpha=0.9,
                     edgecolor="navy",
                     linewidth=2),
            fontsize=12,
            fontweight="bold")

    # Rimuovi spines superiore e destro per aspetto più pulito
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(1.5)
    ax.spines["bottom"].set_linewidth(1.5)

    # Tick più visibili
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.tight_layout()
    plt.savefig(f"risultati_grafici/{nome_rete}.png")

    # Mostra il grafico inline nel notebook
    plt.show()

## 6. Funzioni Specifiche del Min-Max Ant System

In [ ]:
def inizializza_feromone_mmas(G, tau_max):
    """
    MMAS: Inizializza tutti gli archi con il valore massimo di feromone
    per favorire l'esplorazione iniziale
    """
    for u, v in G.edges():
        G[u][v]["pheromone"] = tau_max


def applica_limiti_feromone_mmas(G, tau_min, tau_max):
    """
    MMAS: Applica i limiti di feromone a tutti gli archi
    Mantiene i valori tra tau_min e tau_max per evitare stagnazione
    """
    for u, v in G.edges():
        feromone_attuale = G[u][v]["pheromone"]
        G[u][v]["pheromone"] = max(tau_min, min(tau_max, feromone_attuale))


def evaporazione_feromone_mmas(G, rho):
    """
    MMAS: Applica l'evaporazione del feromone a tutti gli archi del grafo.
    Formula: τ(i,j) = (1 - rho) * τ(i,j)
    Successivamente applica i limiti tau_min e tau_max
    """
    for u, v in G.edges():
        G[u][v]["pheromone"] *= (1 - rho)
    
    # Applica i limiti dopo l'evaporazione
    applica_limiti_feromone_mmas(G, tau_min, tau_max)


def deposito_feromone_miglior_formica_mmas(G, miglior_cammino, miglior_flusso, Q, tau_min, tau_max):
    """
    MMAS: REGOLA FONDAMENTALE - Solo la formica con il miglior percorso può depositare feromone.
    Il deposito è proporzionale alla qualità della soluzione ma limitato per evitare convergenza troppo rapida.
    """
    if not miglior_cammino or miglior_flusso <= 0:
        return
    
    # Deposito feromone SOLO sul percorso della miglior formica
    for i in range(len(miglior_cammino) - 1):
        u, v = miglior_cammino[i], miglior_cammino[i + 1]
        if G.has_edge(u, v):
            # Deposito più conservativo per convergenza graduale
            deposito = Q / len(miglior_cammino)  # Normalizzato per lunghezza cammino
            G[u][v]["pheromone"] += deposito
    
    # Applica i limiti dopo il deposito per mantenere τ_min ≤ τ ≤ τ_max
    applica_limiti_feromone_mmas(G, tau_min, tau_max)


def aggiornamento_feromone_mmas(G, miglior_cammino_globale, miglior_flusso_globale, 
                               miglior_cammino_iterazione, miglior_flusso_iterazione, 
                               iterazione, rho, Q, tau_min, tau_max):
    """
    MMAS: Gestisce l'aggiornamento completo del feromone.
    REGOLA FONDAMENTALE: Solo la formica con il miglior percorso può depositare feromone.
    Strategia: usa sempre la miglior soluzione dell'iterazione per mantenere diversificazione.
    """
    # Fase 1: Evaporazione su tutti gli archi
    evaporazione_feromone_mmas(G, rho)
    
    # Fase 2: Deposito SOLO dalla miglior formica dell'iterazione
    # Questa strategia mantiene maggiore diversificazione e convergenza più graduale
    if miglior_cammino_iterazione and miglior_flusso_iterazione > 0:
        deposito_feromone_miglior_formica_mmas(G, miglior_cammino_iterazione, 
                                              miglior_flusso_iterazione, Q, tau_min, tau_max)


def stampa_statistiche_feromone_mmas(G, iterazione):
    """
    MMAS: Stampa statistiche sui livelli di feromone per monitorare l'algoritmo.
    Include informazioni sui limiti tau_min e tau_max.
    """
    feromoni = [G[u][v]["pheromone"] for u, v in G.edges()]
    if feromoni:
        min_fer = min(feromoni)
        max_fer = max(feromoni)
        avg_fer = np.mean(feromoni)
        print(f"   🧪 MMAS Feromone - Min: {min_fer:.3f}, Max: {max_fer:.3f}, Avg: {avg_fer:.3f}")
        print(f"   📊 Limiti: τ_min={tau_min:.3f}, τ_max={tau_max:.3f}")


def reset_grafo_mmas(G_originale, tau_max):
    """
    MMAS: Resetta il grafo alle condizioni iniziali per un nuovo run.
    Inizializza tutti gli archi con tau_max per favorire esplorazione iniziale.
    """
    G = G_originale.copy()
    # Resetta capacità e inizializza feromone con tau_max
    for u, v in G.edges():
        G[u][v]["capacity"] = G[u][v]["eta"]  # Ripristina capacità originale
        G[u][v]["pheromone"] = tau_max  # MMAS: Inizia con tau_max
    return G

## 7. Algoritmo MMAS - Singolo Run


In [ ]:
def MMAS_single_run(
    G,
    sorgente,
    pozzo,
    flusso_max_teorico,
    iter_max=3000,  # Ridotto per evitare convergenza troppo veloce
    alpha=1,
    beta=5,  # Maggiore importanza dell'euristica
    rho=0.02,  # Evaporazione molto lenta
    Q=100.0,
    tau_min=0.1,
    tau_max=6.0,
    verbose=True,
):
    """
    MMAS: Esegue un singolo run dell'algoritmo Min-Max Ant System.
    Restituisce: (flusso_totale, flussi_per_iterazione, iterazione_miglior_soluzione, valutazioni_funzione)
    """
    flusso_totale = 0.0
    flussi_per_iterazione = []
    
    # Variabili per tracciare la miglior soluzione globale
    miglior_flusso_globale = 0.0
    miglior_cammino_globale = []
    iterazione_miglior_soluzione = 0
    valutazioni_funzione = 0

    # MMAS: Inizializza tutti gli archi con tau_max
    inizializza_feromone_mmas(G, tau_max)

    if verbose:
        print("🐜 Inizio algoritmo Min-Max Ant System...")
        print("=" * 60)

    for iterazione in range(1, iter_max + 1):
        formiche_attive = 0
        flussi_iterazione = 0.0
        
        # Variabili per tracciare la miglior soluzione dell'iterazione
        miglior_flusso_iterazione = 0.0
        miglior_cammino_iterazione = []
        cammini_validi = []

        # Numero formiche = numero archi uscenti dalla sorgente
        num_formiche = len(list(G.out_edges(sorgente)))

        if verbose:
            print(f"\n🔄 Iterazione {iterazione} - Inizio con {num_formiche} formiche")

        for formica in range(num_formiche):
            u = sorgente
            cammino = [u]
            flussi = []
            visitati = set([u])

            # Costruzione cammino probabilistico
            while u != pozzo:
                vettore = proportional_transition_rule(G, u, alpha, beta)
                if not vettore:
                    break

                # Escludi nodi già visitati per evitare cicli
                nodi = [
                    x["nodo_uscente"]
                    for x in vettore
                    if x["nodo_uscente"] not in visitati
                ]
                prob = [
                    x["probabilità"]
                    for x in vettore
                    if x["nodo_uscente"] not in visitati
                ]

                if not nodi:
                    break

                # Normalizza manualmente le probabilità
                prob = np.array(prob)
                prob = prob / prob.sum()

                prossimo = np.random.choice(nodi, p=prob)
                cammino.append(prossimo)
                flussi.append(G[u][prossimo]["capacity"])
                visitati.add(prossimo)
                u = prossimo

            # Se ha raggiunto il pozzo, aggiorna flusso e capacità
            if u == pozzo:
                formiche_attive += 1
                """
                *** VALUTAZIONE FUNZIONE OBIETTIVO ***
                Qui calcoliamo la qualità della soluzione trovata dalla formica
                """
                flusso_cammino = min(flussi)  # ← QUESTA È UNA VALUTAZIONE!
                valutazioni_funzione += 1     # ← Incrementa il contatore delle valutazioni

                """
                La valutazione ci dice: "Quanto flusso può passare attraverso questo cammino?"
                È il collo di bottiglia del percorso trovato dalla formica
                """
                
                flusso_totale += flusso_cammino
                flussi_iterazione += flusso_cammino

                # Memorizza il cammino per l'eventuale aggiornamento del feromone
                cammini_validi.append(
                    {"cammino": cammino.copy(), "flusso": flusso_cammino}
                )

                # Aggiorna il miglior cammino dell'iterazione
                if flusso_cammino > miglior_flusso_iterazione:
                    miglior_flusso_iterazione = flusso_cammino
                    miglior_cammino_iterazione = cammino.copy()

                # Aggiorna il miglior cammino globale
                if flusso_cammino > miglior_flusso_globale:
                    miglior_flusso_globale = flusso_cammino
                    miglior_cammino_globale = cammino.copy()
                    iterazione_miglior_soluzione = iterazione

                # Aggiorna capacità residua sugli archi del cammino
                for i in range(len(cammino) - 1):
                    a, b = cammino[i], cammino[i + 1]
                    G[a][b]["capacity"] -= flusso_cammino
                    G[a][b]["capacity"] = max(0.0, G[a][b]["capacity"])

                    """
                    Impostare Verbose uguale a True nel blocco 8  per stampare il cammino e il flusso e tutti gli aggiornamenti relativi
                    """
                    
                    
                if verbose:
                    print(
                        f"   🐜 Formica {formica+1}: {cammino} | Flusso: {flusso_cammino:.2f}"
                    )
            else:
                if verbose:
                    print(f"   ❌ Formica {formica+1} bloccata: {cammino}")

      
        if verbose:
            if miglior_cammino_iterazione:
                print(f"   🐜 MMAS: Deposito dalla MIGLIOR FORMICA ITERAZIONE (flusso: {miglior_flusso_iterazione:.2f})")
            else:
                print(f"   ❌ MMAS: Nessun deposito (nessuna formica valida)")
            
        aggiornamento_feromone_mmas(
            G, miglior_cammino_globale, miglior_flusso_globale,
            miglior_cammino_iterazione, miglior_flusso_iterazione,
            iterazione, rho, Q, tau_min, tau_max
        )

        # Salva il flusso totale accumulato fino a questa iterazione
        flussi_per_iterazione.append(flusso_totale)

        if verbose:
            print(
                f"➡️ Iterazione {iterazione}: {formiche_attive} cammini validi, "
                f"flusso iterazione: {flussi_iterazione:.2f}, flusso totale: {flusso_totale:.2f}"
            )
            print(f"   🏆 Miglior flusso globale: {miglior_flusso_globale:.2f}")
            stampa_statistiche_feromone_mmas(G, iterazione)

        # Controllo condizione di arresto - flusso massimo teorico raggiunto
        if abs(flusso_totale - flusso_max_teorico) < 1e-6:
            if verbose:
                print(
                    f"\n✅ Flusso massimo teorico raggiunto: {flusso_totale:.2f} "
                    f"(teorico: {flusso_max_teorico:.2f})"
                )
            break

        if formiche_attive == 0:
            if verbose:
                print("\n❌ Nessuna formica ha trovato un cammino. Arresto.")
            break

    return (
        flusso_totale,
        flussi_per_iterazione,
        iterazione_miglior_soluzione,
        valutazioni_funzione,
    )

## 8. Esperimento Completo Multi-Run

In [ ]:
def MMAS_multi_run(
    grafo_originale, sorgente, pozzo, flusso_max_teorico, nome_rete, runs=10, iter_max=3000
):
    """
    MMAS: Esegue l'esperimento completo con 10 runs e calcola le statistiche richieste.
    """
    print(f"🧪 Inizio esperimento Min-Max Ant System con {runs} runs...")
    print("=" * 80)

    # Liste per raccogliere i risultati
    flussi_massimi = []
    iterazioni_miglior_soluzione = []
    valutazioni_funzione_totali = []
    tutti_flussi_iterazione = []

    # Semi diversi per ogni run
    semi = [765344, 748813, 737526, 842352, 5049334, 886324, 447103, 1520231, 229524, 702673]

    for run in range(runs):
        print(f"\n🔄 RUN {run + 1}/{runs} - Seme: {semi[run]}")
        print("-" * 40)

        # Imposta il seme per la riproducibilità
        np.random.seed(semi[run])
        random.seed(semi[run])

        # Resetta il grafo per il nuovo run con inizializzazione MMAS
        G = reset_grafo_mmas(grafo_originale, tau_max)

        # Esegue il singolo run MMAS
        flusso_finale, flussi_iterazione, iter_miglior, valutazioni = MMAS_single_run(
            G,
            sorgente,
            pozzo,
            flusso_max_teorico,
            iter_max,
            alpha,
            beta,
            rho,  # rho più basso per MMAS
            Q,
            tau_min,
            tau_max,
            verbose=False,
        )

        # Memorizza i risultati
        flussi_massimi.append(flusso_finale)
        iterazioni_miglior_soluzione.append(iter_miglior) 
        valutazioni_funzione_totali.append(valutazioni)
        tutti_flussi_iterazione.append(flussi_iterazione)

        print(
            f"✅ Run {run + 1} completato: Flusso = {flusso_finale:.2f}, "
            f" Valutazioni = {valutazioni} , Numero iterazioni = {len(flussi_iterazione)-1}"
        )

    # Calcola le statistiche richieste
    best = max(flussi_massimi)
    mean = statistics.mean(flussi_massimi)
    std_dev = statistics.stdev(flussi_massimi) if len(flussi_massimi) > 1 else 0
    avg_iterations = statistics.mean(iterazioni_miglior_soluzione)
    avg_evaluations = statistics.mean(valutazioni_funzione_totali)

    statistiche = {
        "best": best,
        "mean": mean,
        "std_dev": std_dev,
        "avg_iterations": avg_iterations,
        "avg_evaluations": avg_evaluations,
    }

    # Stampa risultati finali
    print("\n" + "=" * 80)
    print("📊 RISULTATI FINALI - STATISTICHE MIN-MAX ANT SYSTEM (10 RUNS)")
    print("=" * 80)
    print(f"(1) Valore massimo del flusso trovato (best): {best:.2f}")
    print(f"(2) Media dei valori massimi (mean): {mean:.2f}")
    print(f"(3) Deviazione standard (std dev): {std_dev:.2f}")
    print(f"(4) Numero medio di iterazioni per migliore soluzione: {avg_iterations:.1f}")
    print(f"(5) Numero medio di valutazioni funzione obiettivo: {avg_evaluations:.1f}")
    print("=" * 80)

    # Stampa tutti i risultati individuali
    print("\n📋 RISULTATI INDIVIDUALI:")
    for i, (flusso, iter_miglior, valutazioni) in enumerate(
        zip(flussi_massimi, iterazioni_miglior_soluzione, valutazioni_funzione_totali)
    ):
        print(
            f"Run {i+1}: Flusso = {flusso:.2f}, "
            f"Valutazioni = {valutazioni}"
        )

    # Genera il grafico principale con nome della rete
    print(f"\n📊 Generazione grafico Min-Max Ant System Convergence...")
    
    
    disegna_grafico_flusso_multi_run(
        tutti_flussi_iterazione, flusso_max_teorico, statistiche, nome_rete
    )

    

    return statistiche

## 9. Esecuzione dell'Esperimento

In [ ]:
print("🔧 Min-Max Ant System per Maximum Flow Problem")
print("=" * 60)

# Inizializza il grafo originale
grafo_originale, sorgente, pozzo, nome_rete = inizializzazione_grafo()


# Calcola il flusso massimo teorico
flusso_massimo = calcola_flusso_massimo(grafo_originale, sorgente, pozzo)
print(f"\nFlusso massimo stimato (teorico): {flusso_massimo}")

print(f"\n Parametri MMAS:")
print(f"   • α (importanza feromone): {alpha}")
print(f"   • β (importanza euristica): {beta}")
print(f"   • ρ (evaporazione): {rho}")
print(f"   • τ_min: {tau_min}")
print(f"   • τ_max: {tau_max}")
print(f"   • Q (deposito): {Q}")



## 11. Esecuzione dell'Algoritmo MMAS

In [ ]:
statistiche_finali = MMAS_multi_run(
    grafo_originale, sorgente, pozzo, flusso_massimo, nome_rete, runs=10, iter_max=20000
)

import os 
output_dir = "risultati_grafici"
os.makedirs(output_dir, exist_ok=True)
plt.show()


 ## 12. Riepilogo Finale


In [ ]:
print(f"\n🏆 RISULTATI FINALI:")
print(f"   • Best Flusso: {statistiche_finali['best']:.2f}")
print(f"   • Mean Flusso: {statistiche_finali['mean']:.2f}")
print(f"   • Std Dev: {statistiche_finali['std_dev']:.2f}")
print(f"   • Avg Iterazioni: {statistiche_finali['avg_iterations']:.1f}")
print(f"   • Avg Valutazioni: {statistiche_finali['avg_evaluations']:.1f}")